In [1]:
import os
import shutil
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
import torchvision.models as models
from torchvision.io import read_image
from torchvision.transforms import ToTensor, Normalize
from PIL import Image
import pdb
import numpy as np
from tqdm import tqdm
import random
import matplotlib.pyplot as plt

/work/scorreacardo_umass_edu/miniconda3/envs/urbano2/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class_map = {0:'GSD_50cm',
 1:'GSD_65cm',
 2:'GSD_80cm',
 3:'GSD_100cm',
 4:'GSD_124cm',
 5:'GSD_150cm',
 6:'GSD_175cm',
 7:'GSD_200cm',
 8:'GSD_250cm',
 9:'GSD_300cm'}

In [3]:
# Move the model to the appropriate device (e.g., GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# Create an instance of the same model architecture
model = models.resnet18(pretrained=False)
num_classes = 10
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Load the saved model state dictionary
model.load_state_dict(torch.load('/work/scorreacardo_umass_edu/DeepSatGSD/models/trained_model_gsd_resnet18_epochs_3_training_40_perc.pt'))
model.eval()  # Set the model to evaluation mode
# Define the transformation to be applied to the images
transform = torchvision.transforms.Compose([
    ToTensor(),
 
    Normalize([0.46619912981987, 0.4138015806674957, 0.2945951819419861], 
              [0.19115719199180603, 0.1479424238204956, 0.13974712789058685]) 
])
# Path to the folder containing the images for inference
inference_folder = '/gypsum/eguide/projects/scorreacardo/urbano/data/processed/train_img'
inference_folder_mask = '/gypsum/eguide/projects/scorreacardo/urbano/data/processed/train_masks'

# Path to the folder where the images will be copied based on the predicted class
output_folder = '/work/scorreacardo_umass_edu/DeepSatGSD/data/processed/inferece_gep_classified'

# Iterate over the images in the inference folder
for image_name in tqdm(os.listdir(inference_folder)[7289+16876:]):
    image_path = os.path.join(inference_folder, image_name)
    mask_path = os.path.join(inference_folder_mask, image_name)

    # Load and preprocess the image
    image = np.asarray(Image.open(image_path))
    mask = np.asarray(Image.open(mask_path))
    image = image[:, :, :3]
    mask = mask[:, :, :3]
    image = transform(image).float()
    image = image.unsqueeze(0)  # Add batch dimension
    #image = image.to(device, dtype=torch.float32)  # Move the image to GPU
    
    # Perform inference
    with torch.no_grad():
        output = model(image)
    _, predicted_class = torch.max(output, 1)
    
    predicted_class = predicted_class.item()

    # Create the folder if it doesn't exist
    class_folder = os.path.join(output_folder, class_map[predicted_class])
    img_folder = os.path.join(class_folder, 'imgs')
    mask_folder = os.path.join(class_folder, 'masks')
    os.makedirs(class_folder, exist_ok=True)
    os.makedirs(img_folder, exist_ok=True)
    os.makedirs(mask_folder, exist_ok=True)

    # Copy the image to the respective class folder
    shutil.copy(image_path, img_folder)
    # Copy the mask to the respective class folder
    shutil.copy(mask_path, mask_folder)

cuda


100%|██████████| 3381/3381 [33:31<00:00,  1.68it/s] 


In [3]:
import os
import random
import shutil

# Set the paths to your folders
imgs_folder = '/work/scorreacardo_umass_edu/DeepSatGSD/data/processed/inferece_gep_classified/GSD_65cm/imgs'
masks_folder = '/work/scorreacardo_umass_edu/DeepSatGSD/data/processed/inferece_gep_classified/GSD_65cm/masks'
train_folder = '/work/scorreacardo_umass_edu/DeepSatGSD/data/processed/inferece_gep_classified/GSD_65cm/train'
valid_folder = '/work/scorreacardo_umass_edu/DeepSatGSD/data/processed/inferece_gep_classified/GSD_65cm/valid'
train_imgs_folder = os.path.join(train_folder, 'imgs')
train_masks_folder = os.path.join(train_folder, 'masks')
valid_imgs_folder = os.path.join(valid_folder, 'imgs')
valid_masks_folder = os.path.join(valid_folder, 'masks')

# Create the train and valid folders if they don't exist
os.makedirs(train_imgs_folder, exist_ok=True)
os.makedirs(train_masks_folder, exist_ok=True)
os.makedirs(valid_imgs_folder, exist_ok=True)
os.makedirs(valid_masks_folder, exist_ok=True)

# Set the random seed for reproducibility
random.seed(42)

# Get the list of image file names in the imgs folder
all_files = os.listdir(imgs_folder)

# Filter out non-image files based on file extensions
image_files = [file for file in all_files if os.path.splitext(file)[1].lower() in ['.jpg', '.jpeg', '.png']]


# Shuffle the list of image file names
random.shuffle(image_files)

# Define the desired ratio of training to validation images
train_ratio = 0.8  # 80% for training, 20% for validation

# Calculate the number of training images based on the ratio
num_train = int(len(image_files) * train_ratio)

# Split the image and mask files into training and validation sets
train_images = image_files[:num_train]
valid_images = image_files[num_train:]

# Copy the training images and masks to the train folder
for image_file in train_images:
    img_src = os.path.join(imgs_folder, image_file)
    mask_src = os.path.join(masks_folder, image_file)
    shutil.copy(img_src, train_imgs_folder)
    shutil.copy(mask_src, train_masks_folder)

# Copy the validation images and masks to the valid folder
for image_file in valid_images:
    img_src = os.path.join(imgs_folder, image_file)
    mask_src = os.path.join(masks_folder, image_file)
    shutil.copy(img_src, valid_imgs_folder)
    shutil.copy(mask_src, valid_masks_folder)

In [ ]:
import os
from skimage import io

def check_images(folder_path):
    error_images = []

    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        try:
            image = io.imread(file_path)
        except Exception as e:
            print(f"Error reading image '{file_name}': {e}")
            error_images.append(file_name)

    return error_images

# Replace 'your_folder_path' with the actual path to your folder containing images
folder_with_images = '/work/scorreacardo_umass_edu/DeepSatGSD/data/processed/inferece_gep_classified/GSD_65cm/valid/imgs'
images_with_issues = check_images(folder_with_images)

if not images_with_issues:
    print("All images were read successfully.")
else:
    print("The following images had issues reading:")
    for img in images_with_issues:
        print(img)
